In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import torch
import numpy as np

# Define the parent folder path
parent_folder_path = '/content/drive/MyDrive/6389_Ass3/6389Project3Data&ExampleRNN/6389Project3Data'

# Specify file prefix and extension
file_prefix = 'raw_fmri_feature_matrix_'
file_extension = 'txt'

# List to store tensor data for each folder
all_tensor_data = []

# Iterate over both 'AD' and 'CN'
for condition in ['AD', 'CN']:
    for patient_num in range(1, 11):
        # Construct the folder path based on the condition and patient number
        folder_path = os.path.join(parent_folder_path, f'{condition}{patient_num}/fmri_average_signal/')

        # Function to read and concatenate text files in a folder
        def read_and_concatenate_files(folder_path, file_prefix, file_extension):
            data = []

            # List all files in the folder that match the pattern
            files = [f for f in os.listdir(folder_path) if f.startswith(file_prefix) and f.endswith(file_extension)]

            for file in files:
                file_path = os.path.join(folder_path, file)

                # Read the content of the text file
                with open(file_path, 'r') as f:
                    content = f.read().split()

                # Convert the content to a NumPy array
                array_data = np.array(content, dtype=np.float32)

                # Append the array to the list
                data.append(array_data)

            # Stack the list of arrays to create a 2D NumPy array
            concatenated_data = np.vstack(data)

            # Convert the NumPy array to a PyTorch tensor
            tensor_data = torch.tensor(concatenated_data, dtype=torch.float32)

            return tensor_data

        # Call the function to read and concatenate files
        tensor_data = read_and_concatenate_files(folder_path, file_prefix, file_extension)

        # Append the tensor data to the list
        all_tensor_data.append((tensor_data, 1 if condition == 'AD' else 0))  # Assuming binary classification

# Display the shape of the resulting tensors
for tensor_data, label in all_tensor_data:
    print(f"Tensor Shape: {tensor_data.shape}, Label: {label}")
    # numpy_array = tensor_data.numpy()
    # print(numpy_array)

Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 1
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0
Tensor Shape: torch.Size([101, 150]), Label: 0


# **Stacked RNN Model**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
# RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        out = self.sigmoid(out)
        return out

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset

# Assuming all_tensor_data is the list containing tuples of (tensor_data, label)

# Concatenate all tensors into a single tensor
all_data_tensor = torch.stack([tensor_data for tensor_data, _ in all_tensor_data])
all_labels_tensor = torch.tensor([label for _, label in all_tensor_data], dtype=torch.float32)

# Split the data into train and test sets
dataset = TensorDataset(all_data_tensor, all_labels_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the model
input_size = 150  # Size of each time step
hidden_size = 64
output_size = 1
num_layers = 1

model = RNNModel(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# Training Accuracy
model.eval()
with torch.no_grad():
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        predicted_train = (outputs.squeeze() > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

training_accuracy = correct_train / total_train
print(f"StackedRNN Training Accuracy: {training_accuracy}")

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"StackedRNN Test Accuracy: {accuracy}")

StackedRNN Training Accuracy: 1.0
StackedRNN Test Accuracy: 0.75


# **Bidirectional RNN Model**

In [10]:
# Bidirectional RNN Model
class BidirectionalRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(BidirectionalRNNModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # Multiply by 2 for bidirectional
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        out = self.sigmoid(out)
        return out

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset

# Assuming all_tensor_data is the list containing tuples of (tensor_data, label)

# Concatenate all tensors into a single tensor
all_data_tensor = torch.stack([tensor_data for tensor_data, _ in all_tensor_data])
all_labels_tensor = torch.tensor([label for _, label in all_tensor_data], dtype=torch.float32)

# Split the data into train and test sets
dataset = TensorDataset(all_data_tensor, all_labels_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the model
input_size = 150  # Size of each time step
hidden_size = 64
output_size = 1
num_layers = 1

model = BidirectionalRNNModel(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# Training Accuracy
model.eval()
with torch.no_grad():
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        predicted_train = (outputs.squeeze() > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

training_accuracy = correct_train / total_train
print(f"BidirectionalRNN Training Accuracy: {training_accuracy}")

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"BidirectionalRNN Test Accuracy: {accuracy}")

BidirectionalRNN Training Accuracy: 1.0
BidirectionalRNN Test Accuracy: 0.75


# **GRU-based RNN Model**

In [15]:
# GRU Model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GRUModel, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # Multiply by 2 for bidirectional
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        out = self.sigmoid(out)
        return out

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset

# Assuming all_tensor_data is the list containing tuples of (tensor_data, label)

# Concatenate all tensors into a single tensor
all_data_tensor = torch.stack([tensor_data for tensor_data, _ in all_tensor_data])
all_labels_tensor = torch.tensor([label for _, label in all_tensor_data], dtype=torch.float32)

# Split the data into train and test sets
dataset = TensorDataset(all_data_tensor, all_labels_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the model
input_size = 150  # Size of each time step
hidden_size = 64
output_size = 1
num_layers = 1

model = GRUModel(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# Training Accuracy
model.eval()
with torch.no_grad():
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        predicted_train = (outputs.squeeze() > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

training_accuracy = correct_train / total_train
print(f"GRU-based RNN Training Accuracy: {training_accuracy}")

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"GRU-based RNN Test Accuracy: {accuracy}")

GRU-based RNN Training Accuracy: 1.0
GRU-based RNN Test Accuracy: 0.75
